In [1]:
import numpy as np
import tensorflow as tf
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST',one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [43]:
def add_layer(inputs,in_shape,out_shape,layer=None,activation_function=None,dropout=None):
    W = weight_variable(in_shape)
    b = bias_variable(out_shape)
    if layer is "conv":
        h_conv = activation_function(conv2d(inputs,W)+b)
        h_pool = max_pool_2x2(h_conv)
        return h_pool
    wx_plus_b = tf.matmul(inputs,W)+b
    output = activation_function(wx_plus_b)
    if dropout is None:
        return output
    output_dropout = tf.nn.dropout(output,keep_prob)
    return output_dropout

In [44]:
def compute_accuracy(x_v,y_v):
    global prediction
    ypred = sess.run(prediction,feed_dict={xs:x_v,keep_prob:1})
    accurate_counts = tf.equal(tf.arg_max(y_v,1),tf.arg_max(ypred,1))
    accuracy = tf.reduce_mean(tf.cast(accurate_counts,dtype=tf.float32))
    results = sess.run(accuracy,feed_dict={xs:x_v,ys:y_v,keep_prob:1})
    return results

In [45]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [46]:
def conv2d(x,W):
    # stride [1,x_movement,y_movement,1]
    # # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [47]:
def max_pool_2x2(x):
    # stride [1,x_movement,y_movement,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [48]:
#input placeholders
xs = tf.placeholder(tf.float32,[None,784])
ys = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs,[-1,28,28,1])

In [49]:
##conv1 layer##
h_conv_pool_1 = add_layer(x_image,[5,5,1,32],[32],activation_function=tf.nn.relu,layer='conv')
# patch 5x5, in size 1, out size 32
# output size 28x28x32  (from conv2d)
# output size 14x14x32  (from max_pool_2x2)

In [50]:
##conv2 layer##
h_conv_pool_2 = add_layer(h_conv_pool_1,[5,5,32,64],[64],activation_function=tf.nn.relu,layer='conv')
# patch 5x5, in size 32, out size 64
# output size 14x14x64 (from conv2d)
# output size 7x7x64 (from conv2d)

In [51]:
##fc layer 1##
h_conv_pool_2_flat = tf.reshape(h_conv_pool_2,[-1,7*7*64])
fc_1 = add_layer(h_conv_pool_2_flat,[7*7*64,1024],[1024],activation_function=tf.nn.relu,dropout=True)

In [53]:
##fc layer 2##
prediction = add_layer(fc_1,[1024,10],[10],activation_function=tf.nn.softmax)

In [54]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))

In [55]:
training = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [56]:
init = tf.initialize_all_variables()

In [57]:
sess = tf.Session()

In [71]:
sess.run(init)

In [72]:
for i in range(1000):
    batch_x,label_y = mnist.train.next_batch(200)
    sess.run(training,feed_dict={xs:batch_x,ys:label_y,keep_prob:0.5})
    if i%50==0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))

0.1049
0.7941
0.885
0.9139
0.9308
0.9384
0.9429
0.9489
0.9543
0.9568
0.9613
0.9622
0.9645
0.9654
0.9685
0.9692
0.9705
0.9708
0.9719
0.9745
